In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from numpy import cos
from math import log
from scipy.interpolate import interp1d
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

#### As equações do modelo:

$dShdt = \mu_H N -a  b_2  (I_M / N)  S_H - \mu_H S_H\\
dIhdt = a  b_2  (I_M / N)  S_H - \gamma  I_H - \mu_H I_H\\
dRhdt = \gamma  I_H - \mu_H R_H$

$dSmdt = b - a  b_1  (I_H / N)  S_M - \mu S_M \\
dEmdt = a b_1  (I_H / N)  S_M - \mu E_M - b_3 E_M - l E_M\\
dImdt = b_3  E_M  - \mu I_M$

In [2]:
%display typeset

### Analisando o SIR:

In [3]:
var('S_H I_H R_H a b2 I_M N gamma mu_H')

(S_H, I_H, R_H, a, b2, I_M, N, gamma, mu_H)

In [4]:
dShdt = mu_H * N -a * b2 * (I_M / N) * S_H - mu_H * S_H
dIhdt = a * b2 * (I_M / N) * S_H - gamma * I_H - mu_H * I_H
dRhdt = gamma * I_H - mu_H * R_H

#### Calculando o equilíbrio do SIR

In [5]:
solve([dShdt, dIhdt, dRhdt],[S_H, I_H, R_H])

[[S_H == N^2*mu_H/(I_M*a*b2 + N*mu_H), I_H == I_M*N*a*b2*mu_H/(I_M*a*b2*gamma + N*mu_H^2 + (I_M*a*b2 + N*gamma)*mu_H), R_H == I_M*N*a*b2*gamma/(I_M*a*b2*gamma + N*mu_H^2 + (I_M*a*b2 + N*gamma)*mu_H)]]

#### Calculando a Jacobiana do SIR

In [6]:
jack_sir=jacobian([dShdt, dIhdt, dRhdt],[S_H, I_H, R_H])
jack_sir

[-I_M*a*b2/N - mu_H                  0                  0]
[        I_M*a*b2/N      -gamma - mu_H                  0]
[                 0              gamma              -mu_H]

In [7]:
cp_sir = jack_sir.characteristic_polynomial()
cp_sir

x^3 + (I_M*a*b2/N + gamma + 3*mu_H)*x^2 + (I_M*a*b2*gamma/N + 2*I_M*a*b2*mu_H/N + 2*gamma*mu_H + 3*mu_H^2)*x + I_M*a*b2*gamma*mu_H/N + I_M*a*b2*mu_H^2/N + gamma*mu_H^2 + mu_H^3

In [8]:
jack_sir.eigenvalues()

[-(I_M*a*b2 + N*mu_H)/N, -gamma - mu_H, -mu_H]

### Analisando o SEI:

In [9]:
var('S_M E_M I_M b a b1 I_H N mu b3 l')

(S_M, E_M, I_M, b, a, b1, I_H, N, mu, b3, l)

In [10]:
dSmdt = b - a * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M - l * E_M
dImdt = b3 * E_M - mu * I_M

#### Calculando o equilíbrio do SEI

In [11]:
solve([dSmdt, dEmdt, dImdt],[S_M, E_M, I_M])

[[S_M == N*b/(I_H*a*b1 + N*mu), E_M == I_H*a*b*b1/(I_H*a*b1*(b3 + l) + N*mu^2 + (I_H*a*b1 + N*(b3 + l))*mu), I_M == I_H*a*b*b1*b3/(I_H*a*b1*(b3 + l)*mu + N*mu^3 + (I_H*a*b1 + N*(b3 + l))*mu^2)]]

#### Calculando a Jacobiana do SEI

In [12]:
jack_sei=jacobian([dSmdt,dEmdt,dImdt],[S_M,E_M,I_M])
jack_sei

[-I_H*a*b1/N - mu                0                0]
[      I_H*a*b1/N     -b3 - l - mu                0]
[               0               b3              -mu]

In [13]:
cp_sei = jack_sei.characteristic_polynomial()
cp_sei

x^3 + (I_H*a*b1/N + b3 + l + 3*mu)*x^2 + (I_H*a*b1*b3/N + I_H*a*b1*l/N + 2*I_H*a*b1*mu/N + 2*b3*mu + 2*l*mu + 3*mu^2)*x + I_H*a*b1*b3*mu/N + I_H*a*b1*l*mu/N + I_H*a*b1*mu^2/N + b3*mu^2 + l*mu^2 + mu^3

In [14]:
jack_sei.eigenvalues()

[-(I_H*a*b1 + N*mu)/N, -b3 - l - mu, -mu]